# Parte 4: Funciones Adicionales y Fechas

In [3]:
!pip install -q pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [20]:
df = spark.read.csv('ventas.csv', inferSchema=True, header=True)
df.show(4)

+---------+-------+-----+------+
| Compania|Persona|Meses|Ventas|
+---------+-------+-----+------+
|   Guguel|    Sam|    4| 200.5|
|   Guguel| Carlos|   12| 120.1|
|   Guguel|  Frank|    6| 340.0|
|Maikrosof| Teresa|   19| 600.9|
+---------+-------+-----+------+
only showing top 4 rows



## 1. Funciones

Más información en: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [13]:
# Ejemplos de funciones
from pyspark.sql.functions import avg, stddev, countDistinct, round

Conteo de elementos distintos

In [10]:
df.select(countDistinct("Compania")).show()

+------------------------+
|count(DISTINCT Compania)|
+------------------------+
|                       4|
+------------------------+



Operaciones estadísticas como `stddev`, `avg`. Se puede utilizar `alias` para asignar un nombre más adecuado a la columna resultante y `round` para redondear.

In [30]:
# Desviación estándar de las ventas
df.select(round(stddev("Ventas"), 2).alias("Desv ventas"),
          round(avg('Ventas'),2).alias('Media ventas')
          ).show()

+-----------+------------+
|Desv ventas|Media ventas|
+-----------+------------+
|      250.2|      360.94|
+-----------+------------+



Para el manejo de cadenas de caracteres se tiene algunas de las siguientes funciones

In [19]:
from pyspark.sql.functions import concat, substring, length, trim, lower, upper, replace

df.select(concat("Compania", "Persona").alias("Concat"),
          substring("Compania", 1, 3).alias("Substring"),
          length("Compania").alias("Longitud"),
          lower("Compania").alias("minúscula"),
          upper("Compania").alias("mayúscula"),
).show()

+----------------+---------+--------+---------+---------+
|          Concat|Substring|Longitud|minúscula|mayúscula|
+----------------+---------+--------+---------+---------+
|       GuguelSam|      Gug|       6|   guguel|   GUGUEL|
|    GuguelCarlos|      Gug|       6|   guguel|   GUGUEL|
|     GuguelFrank|      Gug|       6|   guguel|   GUGUEL|
| MaikrosofTeresa|      Mai|       9|maikrosof|MAIKROSOF|
|    MaikrosofAmy|      Mai|       9|maikrosof|MAIKROSOF|
|MaikrosofVanessa|      Mai|       9|maikrosof|MAIKROSOF|
|    FeisbukCarla|      Fei|       7|  feisbuk|  FEISBUK|
|     FeisbukSara|      Fei|       7|  feisbuk|  FEISBUK|
|        ApolJuan|      Apo|       4|     apol|     APOL|
|       ApolLinda|      Apo|       4|     apol|     APOL|
|      ApolMiguel|      Apo|       4|     apol|     APOL|
|   ApolChristian|      Apo|       4|     apol|     APOL|
+----------------+---------+--------+---------+---------+



Algunas funciones matemáticas aplicables por columna son las siguientes

In [29]:
from pyspark.sql.functions import abs, round, ceil, floor, exp, log

df.select(abs("Ventas").alias("Valor_absoluto"),
          round("Ventas", 2).alias("Redondeo"),
          ceil("Ventas").alias("Ceil"),
          floor("Ventas").alias("Floor"),
          exp("Ventas").alias("Exp"),
          round(log("Ventas"),5).alias("Log")
          ).show(5)

+--------------+--------+----+-----+--------------------+-------+
|Valor_absoluto|Redondeo|Ceil|Floor|                 Exp|    Log|
+--------------+--------+----+-----+--------------------+-------+
|         200.5|   200.5| 201|  200|1.191361665303007...|5.30081|
|         120.1|   120.1| 121|  120|1.441342778710987E52|4.78832|
|         340.0|   340.0| 340|  340|4.572185553551339...|5.82895|
|         600.9|   600.9| 601|  600| 9.2801324706254E260|6.39843|
|         124.4|   124.4| 125|  124|1.062266654414031E54| 4.8235|
+--------------+--------+----+-----+--------------------+-------+
only showing top 5 rows



La instrucción `when` asigna un valor condicionalmente

In [39]:
from pyspark.sql.functions import when

df.select(round(df['Ventas']),
          when(df['Ventas'] >= 500, 'A').otherwise('B').alias('reemplazo'),
).show()

+----------------+---------+
|round(Ventas, 0)|reemplazo|
+----------------+---------+
|           201.0|        B|
|           120.0|        B|
|           340.0|        B|
|           601.0|        A|
|           124.0|        B|
|           243.0|        B|
|           871.0|        A|
|           350.0|        B|
|           250.0|        B|
|           131.0|        B|
|           750.0|        A|
|           350.0|        B|
+----------------+---------+



In [41]:
from pyspark.sql.functions import when, col

df.withColumn('Compania_num',
              when(col('Compania') == 'Guguel', 0)
              .when(col('Compania') == 'Feisbuk', 1)
              .when(col('Compania') == 'Maikrosof', 2)
              .otherwise(3)
).show()

+---------+---------+-----+------+------------+
| Compania|  Persona|Meses|Ventas|Compania_num|
+---------+---------+-----+------+------------+
|   Guguel|      Sam|    4| 200.5|           0|
|   Guguel|   Carlos|   12| 120.1|           0|
|   Guguel|    Frank|    6| 340.0|           0|
|Maikrosof|   Teresa|   19| 600.9|           2|
|Maikrosof|      Amy|   14| 124.4|           2|
|Maikrosof|  Vanessa|   25| 243.2|           2|
|  Feisbuk|    Carla|   37| 870.7|           1|
|  Feisbuk|     Sara|    8| 350.1|           1|
|     Apol|     Juan|    5| 250.0|           3|
|     Apol|    Linda|    1| 130.7|           3|
|     Apol|   Miguel|    9| 750.3|           3|
|     Apol|Christian|   10| 350.4|           3|
+---------+---------+-----+------+------------+



## 2. Fechas



In [42]:
df = spark.read.csv('covid.csv', inferSchema=True, header=True)
df.show(4)

+-----------+-----------+--------------------+--------------------+-------+----------+----+------------------+--------+-----------------+------------------+-----------+---------------+----------------------------+---------------------+------------+------------+---------------+-------------------------+-------------------+---------------------+-------------+----------------+--------------------+----+-----------+---------------------+-----------------+---------------------+-----------------+---------------------+------------+-----------------+---------------------+------------+-----------------+---------------------+------------+-----------------+---------------------+------------+-----------------+
|eess_renaes|eess_diresa|            eess_red|         eess_nombre|id_eess|id_persona|sexo|fecha_ingreso_hosp|flag_uci|fecha_ingreso_uci|fecha_ingreso_ucin|con_oxigeno|con_ventilacion|fecha_segumiento_hosp_ultimo|evolucion_hosp_ultimo|fecha_dosis1|fecha_dosis2|cdc_positividad|cdc_fecha_fallec

In [43]:
df.select(df['fecha_ingreso_hosp'] ).show(5)

+------------------+
|fecha_ingreso_hosp|
+------------------+
|         1/01/2021|
|        17/04/2021|
|        22/03/2021|
|        10/05/2021|
|         7/08/2020|
+------------------+
only showing top 5 rows



Se debe adecuar el formato a `DateTime`. Para realizar esta conversión se puede usar la función `to_date`. Para más información sobre los formatos ver: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [50]:
from pyspark.sql.functions import to_date

# Creación de una columna llamada "ingreso" con formato de fecha "adecuado"
df = df.withColumn('ingreso', to_date(df['fecha_ingreso_hosp'], 'd/M/yyyy'))

df.select(['fecha_ingreso_hosp', 'ingreso']).show(5)

+------------------+----------+
|fecha_ingreso_hosp|   ingreso|
+------------------+----------+
|         1/01/2021|2021-01-01|
|        17/04/2021|2021-04-17|
|        22/03/2021|2021-03-22|
|        10/05/2021|2021-05-10|
|         7/08/2020|2020-08-07|
+------------------+----------+
only showing top 5 rows



In [59]:
from pyspark.sql.functions import year, month, dayofmonth, dayofyear, weekofyear

# Mostrar año, mes, día, semana, por separado
df.select(year(df['ingreso']).alias("año"),
          month(df['ingreso']).alias("mes"),
          dayofmonth(df['ingreso']).alias("día"),
          weekofyear(df['ingreso']).alias("semana"),
          dayofyear(df['ingreso']).alias("día del año")
          ).show(5)

+----+---+---+------+-----------+
| año|mes|día|semana|día del año|
+----+---+---+------+-----------+
|2021|  1|  1|    53|          1|
|2021|  4| 17|    15|        107|
|2021|  3| 22|    12|         81|
|2021|  5| 10|    19|        130|
|2020|  8|  7|    32|        220|
+----+---+---+------+-----------+
only showing top 5 rows



### Ejemplo: agrupamiento con fechas

Se desea conocer qué años están presentes en el conjunto de datos. Se usará `distinct` para recuperar los valores distintos de años

In [63]:
df.select(year(df['ingreso'])).distinct().show()

+-------------+
|year(ingreso)|
+-------------+
|         2023|
|         2022|
|         2020|
|         2021|
+-------------+



Se desea saber cuántos ingresos ocurrieron en cada año

In [65]:
df.groupBy(year(df['ingreso'])).count().show()

+-------------+-----+
|year(ingreso)|count|
+-------------+-----+
|         2023|    5|
|         2022|  127|
|         2020|  269|
|         2021|  597|
+-------------+-----+



Se desea analizar en qué mes ocurrió la mayor cantidad de ingresos, independientemente del año

In [68]:
df.groupBy(month(df['ingreso']).alias('mes')) \
  .count() \
  .withColumnRenamed('count', 'total') \
  .orderBy('mes') \
  .show()

+---+-----+
|mes|total|
+---+-----+
|  1|  100|
|  2|   85|
|  3|  100|
|  4|  137|
|  5|  128|
|  6|  103|
|  7|   85|
|  8|   93|
|  9|   36|
| 10|   41|
| 11|   44|
| 12|   46|
+---+-----+



Se desea saber cuántos internamientos existen por año y por departamento de domicilio, y mostrar los 10 primeros ordenados de mayor a menor

In [78]:
from pyspark.sql.functions import desc

df.groupBy(year(df['ingreso']), df['dep_domicilio']) \
  .count() \
  .orderBy(desc('count')) \
  .show(10)

+-------------+-------------+-----+
|year(ingreso)|dep_domicilio|count|
+-------------+-------------+-----+
|         2021|        PIURA|  142|
|         2021|        TACNA|  116|
|         2021|         LIMA|   81|
|         2021|   SAN MARTIN|   66|
|         2021|      UCAYALI|   63|
|         2020|   SAN MARTIN|   60|
|         2020|         LIMA|   53|
|         2020|        PIURA|   41|
|         2022|        PIURA|   39|
|         2020|      UCAYALI|   35|
+-------------+-------------+-----+
only showing top 10 rows

